In [1]:
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
#from application.app.folder.averaging.py import averaging.py

import sys
sys.path.insert(1, '../tools/')
import iosubdomains as ios

----------

In [9]:
n = 0
path = f"../../../net/alpha/exports/sciences/roullet/Nyles/plume_ref4/plume_ref4_{n:02d}_hist.nc"
print(path)

../../../net/alpha/exports/sciences/roullet/Nyles/plume_ref4/plume_ref4_00_hist.nc


In [10]:
b_00 = ios.Variable(path, 'b')


TypeError: not all arguments converted during string formatting

In [20]:
b_00.check_sanity

<bound method Variable.check_sanity of template    : ../../data/Nyles/plume_ref4/plume_ref4_0%s_hist.nc
variable    : b
dimensions  : ('t', 'z', 'y', 'x')
global shape: (145, 16, 32, 32)
partition   : {'x': 2, 'y': 2, 'z': 1}>

In [1]:
def horizontal_avg(field, yidx=slice(None), xidx=slice(None)):
    """

    Do a horizontal average

    Parameters
    ----------
    field : array,  2D or 3D
         the field that you want to average

    xidx, yidx : slice, optional
         ranging over the interior points, excluding the halo
         default is all points


    Returns
    -------
    res : float or 1D array
       the horizontal average

    """
    if len(field.shape) == 2:
        res = np.mean(field[yidx, xidx])
    elif len(field.shape) == 3:
        res = np.mean(np.mean(field[:, yidx, xidx], axis=2), axis=1)
    else:
        raise ValueError("field should be a 2D or 3D array")
    return res